In [31]:
from scipy.stats import norm
import numpy as np
import os

var("r K s v t v0") # vars used in both models

(r, K, s, v, t, v0)

#### Expression of mis-pricing function $\delta$


In [32]:
density(x) = 1/sqrt(2*pi)*exp(-x^2/2) # Gaussian density
sigma = sqrt(v0)
d1 = (log(s / K) + (r + sigma ^ 2 / 2) * t) / (sigma * sqrt(t))
Gamma = density(d1) / (sigma * s * sqrt(t))
delta0 = 0.5 * (v - sigma ** 2) * s ** 2 * Gamma # mis-pricing of using bs model

#### Black-Scholes formula and update functions for delta

Notice that here we use numerical calculation for black-scholes model, because there's no need to use symbolic calculation for it, we have defined expressions of $\delta$ before.


In [33]:
var("kappa theta sig rho")  # vars used only in heston model


def bs(S, K, T, r, sigma):
    """Numerical calculation of Black-Scholes formula"""
    N = norm.cdf
    d1 = (np.log(S / K) + (r + sigma**2 / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r * T) * N(d2)


def update_delta(f):
    """infinitesimal generator"""
    ft = diff(f, t)
    fs = diff(f, s)
    fs2 = diff(fs, s)
    fv = diff(f, v)
    fv2 = diff(fv, v)
    fvs = diff(fv, s)
    tmp1 = -ft + r * s * fs + kappa * (theta - v) * fv
    tmp2 = (v * s**2 * fs2 + sig**2 * v * fv2) / 2
    tmp3 = rho * sig * v * s * fvs
    return tmp1 + tmp2 + tmp3 - r * f

Now we can use `update_delta` to calculate $\delta_i$ in KM's paper


In [34]:
%%time
delta1 = update_delta(delta0).simplify_full()
delta2 = update_delta(delta1).simplify_full()
delta3 = update_delta(delta2).simplify_full()
delta4 = update_delta(delta3)
deltas = [delta0, delta1, delta2, delta3, delta4]

CPU times: user 15.3 s, sys: 238 ms, total: 15.5 s
Wall time: 14.7 s


#### I/O

Indeed we can save our computation results to a file and load them later. This is useful when we want to evaluate them in a different notebook or in a different session.

In [35]:
if not os.path.exists("sage_output"):
    os.makedirs("sage_output")
for i in range(len(deltas)):
    deltas[i].save(f"./sage_output/delta{i}")

In [36]:
deltas = []
if os.path.exists("sage_output"):
    files = os.listdir("sage_output")
    if len(files) == 5:
        files.sort()
        for i in range(len(files)):
            deltas.append(load(f"./sage_output/delta{i}"))

#### Approximation formula

Now we can wrap all the calculations into one function


In [37]:
def approx(bs, delta, n, params):
    """Approximate option price using the first n terms of the Ito-Taylor series"""
    t, s, v, r, K, v0, kappa, theta, sig, rho = params
    deltas_expression = sum(
        delta[i] * t ** (i + 1) / factorial(i + 1) for i in range(n + 1)
    )
    deltas_numerical = numerical_approx(
        deltas_expression(
            t=t, s=s, v=v, r=r, K=K, v0=v0, kappa=kappa, theta=theta, sig=sig, rho=rho
        )
    )
    bs_numerical = bs(S=s, K=K, T=t, r=r, sigma=np.sqrt(v0))
    return bs_numerical + deltas_numerical

In sagemath, another way is to construct a transform expressions into a form where they can evaluated quickly. However, in our test it's slower than the previous method.

In [38]:
def approx2(bs, delta, n, params):
    t, s, v, r, K, v0, kappa, theta, sig, rho = params
    deltas_expression = sum(
        delta[i] * t ** (i + 1) / factorial(i + 1) for i in range(n + 1)
    )
    fast_delta = fast_callable(
        deltas_expression,
        vars=("t", "s", "v", "r", "K", "v0", "kappa", "theta", "sig", "rho"),
        domain=RR,
    )
    delta_numerical = fast_delta(t, s, v, r, K, v0, kappa, theta, sig, rho)
    bs_numerical = bs(S=s, K=K, T=t, r=r, sigma=np.sqrt(v0))
    return bs_numerical + delta_numerical

#### Evaluate the approximation


In [39]:
kappa = 0.1456
theta = 0.5172
sig = 0.5786
r = 0
K = 1000
rho = -0.0243
v0 = 0.5172
t = 1 / 12
s = 950
v = 0.5172

In [40]:
%%time
print("calculation of panel A in table 1: \n")
for i in range(950, 1060, 10):
    params = (t, i, v, r, K, v0, kappa, theta, sig, rho)
    print(approx(bs, deltas, 4, params))

calculation of panel A in table 1: 

57.84489054110935
62.37375983196581
67.1032999960679
72.03211991278214
77.15835450162963
82.47969295665295
87.993408879552
93.69639186491263
99.58518010991531
105.65599364479358
111.90476780807006
CPU times: user 1.76 s, sys: 14 ms, total: 1.77 s
Wall time: 1.77 s


In [41]:
%%time
s = 1000
print("calculation of panel B in table 1: \n")
for j in np.arange(0.1, 1.2, 0.1):
    v = j
    v0 = j
    params = (t, s, v, r, K, v0, kappa, theta, sig, rho)
    print(approx(bs, deltas, 4, params))

calculation of panel B in table 1: 

36.4824952552777
51.42396003020794
62.905924161421844
72.58339034130205
81.10389934659743
88.80081803753009
95.87260423649352
102.44874976250965
108.61929697660212
114.44986242045158
119.99001814341999
CPU times: user 1.41 s, sys: 12 ms, total: 1.42 s
Wall time: 1.42 s


In [42]:
%%time
print("calculation of panel A in table 1: \n")
for i in range(950, 1060, 10):
    params = (t, i, v, r, K, v0, kappa, theta, sig, rho)
    print(approx2(bs, deltas, 4, params))

calculation of panel A in table 1: 

93.64598276241655
98.64219505836023
103.77634921283627
109.04686171382113
114.45202522438302
119.99001814342002
125.65891405466114
131.45669102247263
137.38124069763592
143.43037720071928
149.60184575493724
CPU times: user 27.2 s, sys: 229 ms, total: 27.4 s
Wall time: 27.5 s
